# Load the data into your project using pandas:
- Import the pandas library: **import pandas as pd**.

In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
print("CSV Readings complete")

CSV Readings complete


In [3]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


# Perform data preprocessing and filtering:

- Replace "\N" with np.nan in each DataFrame: basics.replace({'\\N': np.nan}, inplace=True), akas.replace({'\\N': np.nan}, inplace=True), ratings.replace({'\\N': np.nan}, inplace=True).
- Filter the basics DataFrame based on the provided specifications:

In [4]:
basics = pd.DataFrame(basics).replace({'\\N': np.nan})
akas = pd.DataFrame(akas).replace({'\\N': np.nan})
ratings = pd.DataFrame(ratings).replace({'\\N': np.nan})

print("NaNs were Replaced Succesfully")

NaNs were Replaced Succesfully


In [5]:
basics = basics.dropna(subset=['runtimeMinutes', 'genres'])
basics = basics[basics['titleType'] == 'movie']
basics['startYear'] = pd.to_numeric(basics['startYear'], errors='coerce') # Convert 'startYear' to numeric type
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2021)]
basics = basics[~basics['genres'].str.contains('documentary', case=False)]

### Filter the basics DataFrame to include only US movies based on the akas DataFrame:

In [6]:
akas = akas[(akas['region'] == 'US')]
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [7]:
keepers = basics['tconst'].isin(akas['titleId'])
keepers.head

<bound method NDFrame.head of 34803       True
42384       True
61115       True
67667       True
86799       True
           ...  
9949133     True
9949142     True
9949181    False
9949226     True
9949310    False
Name: tconst, Length: 138660, dtype: bool>

In [8]:
basics = basics[keepers]

In [9]:
keepers2 = ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers2]

In [10]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81775 entries, 34803 to 9949226
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81775 non-null  object 
 1   titleType       81775 non-null  object 
 2   primaryTitle    81775 non-null  object 
 3   originalTitle   81775 non-null  object 
 4   isAdult         81775 non-null  object 
 5   startYear       81775 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  81775 non-null  object 
 8   genres          81775 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.2+ MB


In [11]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501745 entries, 0 to 1322494
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         501745 non-null  object 
 1   averageRating  501745 non-null  float64
 2   numVotes       501745 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.3+ MB


In [12]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1446602 entries, 5 to 36323741
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1446602 non-null  object
 1   ordering         1446602 non-null  int64 
 2   title            1446602 non-null  object
 3   region           1446602 non-null  object
 4   language         3957 non-null     object
 5   types            980406 non-null   object
 6   attributes       46816 non-null    object
 7   isOriginalTitle  1445260 non-null  object
dtypes: int64(1), object(7)
memory usage: 99.3+ MB


# Save the filtered DataFrames to compressed CSV files:

- Create a "Data" folder within your repository if it doesn't already exist.
- Use the to_csv method to save each DataFrame with compression:

In [13]:
basics.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)
akas.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)
ratings.to_csv("Data/title_ratings.csv.gz", compression='gzip', index=False)

In [14]:
print("Check \"Data\" Folder")

Check "Data" Folder
